In [10]:
import os
import shutil
import random

random.seed(42)

# =========================
# PATHS
# =========================
BASE_DIR = r"C:\Users\Umair\Desktop\Data set isic"
OUT_DIR  = r"C:\Users\Umair\Desktop\ISIC_BALANCED_FINAL"

CLASSES = ["Benign", "Malignant", "BCC"]
IMG_EXTS = (".jpg", ".png", ".jpeg")

TARGET_PER_CLASS = 2400  # dictated by Benign

# =========================
# CREATE OUTPUT FOLDERS
# =========================
for split in ["train", "val", "test"]:
    for cls in CLASSES:
        os.makedirs(os.path.join(OUT_DIR, split, cls), exist_ok=True)

# =========================
# LOAD IMAGES
# =========================
def load_images(folder):
    return [
        os.path.join(folder, f)
        for f in os.listdir(folder)
        if f.lower().endswith(IMG_EXTS)
    ]

# =========================
# PROCESS EACH CLASS
# =========================
for cls in CLASSES:
    src_dir = os.path.join(BASE_DIR, cls)
    imgs = load_images(src_dir)

    if len(imgs) < TARGET_PER_CLASS:
        raise ValueError(f"{cls} has fewer than {TARGET_PER_CLASS} images")

    # 1️⃣ Downsample to 2400
    imgs = random.sample(imgs, TARGET_PER_CLASS)

    # 2️⃣ Shuffle
    random.shuffle(imgs)

    # 3️⃣ Split
    n_test = int(0.2 * TARGET_PER_CLASS)          # 480
    n_train_val = TARGET_PER_CLASS - n_test       # 1920
    n_val = int(0.2 * n_train_val)                # 384
    n_train = n_train_val - n_val                 # 1536

    test_imgs = imgs[:n_test]
    val_imgs = imgs[n_test:n_test + n_val]
    train_imgs = imgs[n_test + n_val:]

    # 4️⃣ Copy files
    for src in train_imgs:
        shutil.copy2(src, os.path.join(OUT_DIR, "train", cls, os.path.basename(src)))

    for src in val_imgs:
        shutil.copy2(src, os.path.join(OUT_DIR, "val", cls, os.path.basename(src)))

    for src in test_imgs:
        shutil.copy2(src, os.path.join(OUT_DIR, "test", cls, os.path.basename(src)))

    print(
        f"{cls}: "
        f"{len(train_imgs)} train | "
        f"{len(val_imgs)} val | "
        f"{len(test_imgs)} test"
    )

print("\n✅ DATASET CREATED FROM SCRATCH — BALANCED & READY")


Benign: 1536 train | 384 val | 480 test
Malignant: 1536 train | 384 val | 480 test
BCC: 1536 train | 384 val | 480 test

✅ DATASET CREATED FROM SCRATCH — BALANCED & READY


In [11]:
import os

BASE_DIR = r"C:\Users\Umair\Desktop\ISIC_BALANCED_FINAL"

CLASSES = ["Benign", "Malignant", "BCC"]
SPLITS = ["train", "val", "test"]
IMG_EXTS = (".jpg", ".png", ".jpeg")

EXPECTED_COUNTS = {
    "train": 1536,
    "val": 384,
    "test": 480
}

print("🔍 VERIFYING DATASET STRUCTURE & COUNTS\n")

all_ok = True
split_files = {cls: {} for cls in CLASSES}

for split in SPLITS:
    print(f"📂 {split.upper()}")
    for cls in CLASSES:
        path = os.path.join(BASE_DIR, split, cls)

        if not os.path.exists(path):
            print(f"❌ Missing folder: {path}")
            all_ok = False
            continue

        files = [
            f for f in os.listdir(path)
            if f.lower().endswith(IMG_EXTS)
        ]

        split_files[cls][split] = set(files)

        expected = EXPECTED_COUNTS[split]
        actual = len(files)

        status = "✅" if actual == expected else "❌"
        if actual != expected:
            all_ok = False

        print(f"  {cls}: {actual} images {status}")

    print()

# =========================
# CHECK FOR DATA LEAKAGE
# =========================
print("🔎 CHECKING FOR DATA LEAKAGE\n")

for cls in CLASSES:
    train = split_files[cls]["train"]
    val   = split_files[cls]["val"]
    test  = split_files[cls]["test"]

    overlap_tv = train & val
    overlap_tt = train & test
    overlap_vt = val & test

    if overlap_tv or overlap_tt or overlap_vt:
        print(f"❌ Data leakage detected in {cls}")
        all_ok = False
    else:
        print(f"✅ No overlap in {cls}")

# =========================
# FINAL RESULT
# =========================
print("\n==============================")
if all_ok:
    print("🎉 DATASET VERIFIED SUCCESSFULLY")
    print("✔ Balanced")
    print("✔ Correct splits")
    print("✔ No leakage")
else:
    print("⚠️ DATASET VERIFICATION FAILED")
    print("Check the errors above")
print("==============================")


🔍 VERIFYING DATASET STRUCTURE & COUNTS

📂 TRAIN
  Benign: 1536 images ✅
  Malignant: 1536 images ✅
  BCC: 1536 images ✅

📂 VAL
  Benign: 384 images ✅
  Malignant: 384 images ✅
  BCC: 384 images ✅

📂 TEST
  Benign: 480 images ✅
  Malignant: 480 images ✅
  BCC: 480 images ✅

🔎 CHECKING FOR DATA LEAKAGE

✅ No overlap in Benign
✅ No overlap in Malignant
✅ No overlap in BCC

🎉 DATASET VERIFIED SUCCESSFULLY
✔ Balanced
✔ Correct splits
✔ No leakage
